In [5]:
import pandas as pd

In [11]:
payments = pd.read_excel('./data_1.xlsx')
payments.head()

,ИД дома,ИД подъезда,ИД квартиры,Лицевой счет (ЛС),Услуга,Ставка тарифа,Статус услуги,Дата начала услуги
0,100,1624,43036,43036,ТО МКД,5.2,Активна,2008-01-23
1,100,1624,43037,43037,ТО МКД,5.4,Активна,2008-01-23
2,100,1624,43038,43038,ТО МКД,5.2,Активна,2008-01-23
3,100,1624,43039,43039,ТО МКД,5.2,Активна,2008-01-23
4,100,1624,43040,43040,ТО МКД,5.2,Активна,2008-01-23


In [12]:
accounts = pd.read_excel('./data_2.xlsx')
accounts.head()

,ЛС,Сумма платежа,Дата загрузки,Источник платежа,способ оплаты,Комментарий к платежу
0,1,4.8,2019-04-16,Ручной ввод,Офис,NaN
1,1,14.4,2018-10-17,Ввод из реестра,Сбербанк России,NaN
2,1,14.4,2019-01-21,Ввод из реестра,КС банк,NaN
3,1,32.4,2020-12-11,Ввод из реестра,КС банк,NaN
4,1,56.4,2018-02-09,Ввод из реестра,КС банк,NaN


In [125]:
accounts['Услуга'].unique()

array(['ТО МКД', 'ВБ', 'ТО Дверь', 'Рассрочка за Монтаж ВБ', 'ТО СВОД',
       'Рассрочка за Апгрейд ВБ'], dtype=object)

In [126]:
#смотрим данные таблицы
accounts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49543 entries, 0 to 49542
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   ИД дома             49543 non-null  int64         
 1   ИД подъезда         49543 non-null  int64         
 2   ИД квартиры         49543 non-null  int64         
 3   Лицевой счет (ЛС)   49543 non-null  int64         
 4   Услуга              49543 non-null  object        
 5   Ставка тарифа       49316 non-null  float64       
 6   Статус услуги       49543 non-null  object        
 7   Дата начала услуги  49543 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(4), object(2)
memory usage: 3.0+ MB


Суть проверки:
проверяем, что:
"Одновременно на одной квартире может быть только одна базовая услуга ТО МКД, ТО СВОД или ТО ДВЕРЬ в статусе != Завершена."

In [127]:
selected_services = ['ТО МКД', 'ТО СВОД', 'ТО Дверь']

many_services_1 =accounts.query('`Услуга` in @selected_services')\
                         .query('`Статус услуги` != "Завершена"')\
                         .groupby(['ИД квартиры'])\
                         .agg({'ИД квартиры': 'count'})\
                         .query('`ИД квартиры` > 1')

#Результат проверки: Существуют номера квартир, в которых условие не выполняется:
many_services_1

,ИД квартиры
ИД квартиры,
2506,2
22730,2
23262,2
40749,2
43410,2
43942,2
47526,2
48383,2
49522,2



Предпринятый или предлагаемый способ устранение дефекта данных:
1. Уточнить, являются ли эти статусы приемлимыми, потому что по логике, они вполне реальные
2. Если это всё-таки недопустимые статусы, то удалить строки с неприемлимыми статусами из таблицы
3. Сейчас устранять ничего не буду.

Суть проверки: Если на одном ЛС есть несколько базовых услуг (ТО МКД, ТО СВОД или ТО ДВЕРЬ), то ранее действующая должна быть в статусе Завершена.


In [128]:
selected_services = ['ТО МКД', 'ТО СВОД', 'ТО Дверь']

many_services_2 = accounts.query('`Услуга` in @selected_services')\
                          .groupby(['Лицевой счет (ЛС)'], as_index=False)\
                          .agg({'Услуга':'count'})\
                          .query('`Услуга` > 1')

acc = many_services_2['Лицевой счет (ЛС)']

In [129]:
max_date = accounts.query('`Лицевой счет (ЛС)` in @acc')\
                   .query('`Услуга` in @selected_services')\
                   .groupby('Лицевой счет (ЛС)', as_index=False)\
                   .agg({'Дата начала услуги': 'max'})
max_date

,Лицевой счет (ЛС),Дата начала услуги
0,1,2021-03-01
1,22,2021-03-29
2,26,2019-02-02
3,84,2020-12-24
4,208,2019-04-03
...,...,...
5547,49134,2020-11-01
5548,49136,2020-12-01
5549,49140,2021-05-20
5550,49150,2021-02-02


In [130]:
min_date = accounts.query('`Лицевой счет (ЛС)` in @acc')\
                   .query('`Услуга` in @selected_services')\
                   .query('`Статус услуги` != "Завершена"')\
                   .groupby('Лицевой счет (ЛС)', as_index=False)\
                   .agg({'Дата начала услуги': 'min'})

min_date

,Лицевой счет (ЛС),Дата начала услуги
0,1,2021-03-01
1,22,2021-03-29
2,26,2019-02-02
3,84,2020-12-24
4,208,2019-04-03
...,...,...
5547,49134,2020-11-01
5548,49136,2020-12-01
5549,49140,2021-05-20
5550,49150,2021-02-02


In [131]:
#Результат проверки: Существуют ЛС, для которых условие не выполняется:
max_date.join(min_date, lsuffix=" max", rsuffix=" min").query('`Дата начала услуги max` > `Дата начала услуги min`')

,Лицевой счет (ЛС) max,Дата начала услуги max,Лицевой счет (ЛС) min,Дата начала услуги min
418,2506,2021-11-01,2506,2017-11-14
3652,22730,2019-01-01,22730,2007-02-25
3665,23262,2021-08-19,23262,2019-01-29
4690,40749,2021-07-01,40749,2007-11-12
4750,43410,2019-11-01,43410,2008-02-26
4770,43942,2019-08-01,43942,2008-04-11
5409,47612,2020-01-01,47612,2010-12-22
5434,48140,2019-10-24,48140,2014-07-01
5539,49062,2019-03-28,49062,2019-03-01


Предпринятый или предлагаемый способ устранение дефекта данных:

Уточнить, являются ли эти статусы приемлимыми, потому что по логике, они вполне реальные.

Если это всё-таки недопустимые статусы, то удалить строки с неприемлимыми статусами из таблицы.

Сейчас устранять ничего не буду.

2. Рассчитать для каждого ЛС сумму всех поступивших платежей с учетом возвратов, но исключая списания.

In [132]:
#вспомним нашу таблицу
payments.head()

,ЛС,Сумма платежа,Дата загрузки,Источник платежа,способ оплаты,Комментарий к платежу
0,1,4.8,2019-04-16,Ручной ввод,Офис,NaN
1,1,14.4,2018-10-17,Ввод из реестра,Сбербанк России,NaN
2,1,14.4,2019-01-21,Ввод из реестра,КС банк,NaN
3,1,32.4,2020-12-11,Ввод из реестра,КС банк,NaN
4,1,56.4,2018-02-09,Ввод из реестра,КС банк,NaN


In [133]:
#проверим, какие вообще источники платежа есть
payments['Источник платежа'].unique()

array(['Ручной ввод', 'Ввод из реестра', 'Возврат', 'Сторно минус',
       'Авто. списание', 'Возврат списания', 'Ручное списание',
       'Сторно плюс'], dtype=object)

In [134]:
#создадим таблицу только с нужными нам платежами
#не очень понятно, нужно ли исключать "Возврат списания", ориентируясь на свою логику - я оставила
pays = payments[(payments['Источник платежа'] != 'Авто. списание') & (payments['Источник платежа'] != 'Ручное списание')]
pays['Источник платежа'].unique()

array(['Ручной ввод', 'Ввод из реестра', 'Возврат', 'Сторно минус',
       'Возврат списания', 'Сторно плюс'], dtype=object)

In [135]:
#итог: ответ задачи
task_2 = pays.groupby('ЛС').agg({'Сумма платежа': 'sum'})
task_2

,Сумма платежа
ЛС,
1,182.40
2,236.40
3,230.40
4,236.40
5,222.90
...,...
49146,62.40
49148,31.20
49149,31.20
